In [20]:
# Dependencies
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json

# Dependencies for GDP data
from pandas import ExcelWriter
from pandas import ExcelFile

In [23]:
# File to Load for Olympic Data
olympic_csv = "resources/athlete_events.csv"

# Read .csv File and store into Pandas data frame for Olympic Data
olympic_raw = pd.read_csv(olympic_csv)
pd.options.display.float_format = '{:.2f}'.format
olympic_raw.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.00,180.00,80.00,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.00,170.00,60.00,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.00,nan,nan,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.00,nan,nan,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.00,185.00,82.00,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [24]:
# Load and read .xls file for GDP data
gdp_data = pd.read_excel("Country GDP Final.xls")

In [25]:
# Review and format GDP numbers

#pd.options.display.float_format = '${:,.2f}'.format
gdp_data.head()

,NOC,NOC Region,2000,2001,2002,2003,2004,2005,2006,2007,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,AFG,Afghanistan,nan,nan,4055176933.37,4515563413.83,5226775163.30,6209140044.01,6971286731.72,9747886104.53,...,12439087076.77,15856574731.44,17804280538.11,20001615788.67,20561054090.40,20484873230.21,19907111418.99,19362642266.65,20191764940.16,19362969582.36
1,ALB,Albania,3480355188.60,3922099470.97,4348070165.19,5611492282.87,7184681398.57,8052075642.11,8896073938.31,10677321490.38,...,12044223457.83,11926962834.96,12890867535.56,12319784701.33,12776277648.92,13228244336.32,11386927679.09,11861353752.07,13025064966.58,15102500898.24
2,ALG,Algeria,54786074940.21,54743249368.65,56758113501.17,67866141223.59,85324767230.49,103198442061.08,117030941571.94,134978727825.85,...,137214821177.36,161205065469.31,200015355528.47,209062886917.04,209754763860.68,213808808746.70,165978425168.39,160032930353.76,167390266100.76,173757952824.25
3,AND,Andorra,1434429703.34,1496912751.68,1733116883.12,2398645598.19,2935659299.73,3255789080.96,3543256805.92,4016972351.49,...,3660530702.97,3355695364.24,3442062830.14,3164615186.95,3281585236.33,3350736367.25,2811489408.89,2877311946.90,3013387423.94,3236543909.35
4,ANG,Angola,9129594818.61,8936063723.20,15285594828.42,17812705294.33,23552052407.55,36970918699.25,52381006892.04,65266452081.39,...,70307163678.24,83799496611.20,111789686464.44,128052853643.11,136709862831.19,145712200312.51,116193649124.15,101123851090.46,122123822333.59,105750987618.63


In [52]:
# remove all rows of data for Olympics taking place before the end of WW2 (1945).
olympic_raw_post_war = olympic_raw[olympic_raw.Year > 1945]

# this is our raw dataframe for post ward data.
#olympic_raw_post_war.head()

# split dataframe into two separate dataframes by summer and winter games
# this is our dataframe for summer olympic data, post WW2.
summer_olympics = olympic_raw_post_war[olympic_raw_post_war.Season != "Winter"]
summer_olympics.head()
# this is our dataframe for winter olympic data, post WW2.
#winter_olympics = olympic_raw_post_war[olympic_raw_post_war.Season != "Summer"]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.00,180.00,80.00,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.00,170.00,60.00,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
29,10,"Einar Ferdinand ""Einari"" Aalto",M,26.00,nan,nan,Finland,FIN,1952 Summer,1952,Summer,Helsinki,Swimming,Swimming Men's 400 metres Freestyle,NaN
31,12,Jyri Tapani Aalto,M,31.00,172.00,70.00,Finland,FIN,2000 Summer,2000,Summer,Sydney,Badminton,Badminton Men's Singles,NaN
32,13,Minna Maarit Aalto,F,30.00,159.00,55.50,Finland,FIN,1996 Summer,1996,Summer,Atlanta,Sailing,Sailing Women's Windsurfer,NaN


In [57]:
# Pull just 2016 Olympics
olympics_2016 = summer_olympics.loc[summer_olympics.Year == 2016]
olympics_2016.shape

(13688, 15)

In [64]:
# Drop rows where Medal is NaN
no_null_olympics_2016 = olympics_2016.dropna(how='any')
no_null_olympics_2016.tail()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
269511,134857,Zhu Ting,F,21.00,198.00,78.00,China,CHN,2016 Summer,2016,Summer,Rio de Janeiro,Volleyball,Volleyball Women's Volleyball,Gold
270111,135132,Bojana ivkovi,F,28.00,186.00,72.00,Serbia,SRB,2016 Summer,2016,Summer,Rio de Janeiro,Volleyball,Volleyball Women's Volleyball,Silver
270281,135205,Shakhobiddin Shokirovich Zoirov,M,23.00,169.00,52.00,Uzbekistan,UZB,2016 Summer,2016,Summer,Rio de Janeiro,Boxing,Boxing Men's Flyweight,Gold
270370,135245,Milenko Zori,M,27.00,179.00,73.00,Serbia,SRB,2016 Summer,2016,Summer,Rio de Janeiro,Canoeing,"Canoeing Men's Kayak Doubles, 1,000 metres",Silver
271019,135525,Martin Zwicker,M,29.00,175.00,64.00,Germany,GER,2016 Summer,2016,Summer,Rio de Janeiro,Hockey,Hockey Men's Hockey,Bronze


In [71]:
no_null_olympics_2016 = no_null_olympics_2016.sort_values("Event")
no_null_olympics_2016.head(10)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
247717,124031,Jean-Charles Valladont,M,27.00,180.00,83.00,France,FRA,2016 Summer,2016,Summer,Rio de Janeiro,Archery,Archery Men's Individual,Silver
127976,64487,Ku Bon-Chan,M,23.00,181.00,84.00,South Korea,KOR,2016 Summer,2016,Summer,Rio de Janeiro,Archery,Archery Men's Individual,Gold
63166,32384,Brady Lee Ellison,M,27.00,181.00,86.00,United States,USA,2016 Summer,2016,Summer,Rio de Janeiro,Archery,Archery Men's Individual,Bronze
135437,68149,Lee Seung-Yun,M,21.00,174.00,80.00,South Korea,KOR,2016 Summer,2016,Summer,Rio de Janeiro,Archery,Archery Men's Team,Gold
112682,57081,"Jacob Robert ""Jake"" Linde-Kaminski",M,27.00,178.00,70.00,United States,USA,2016 Summer,2016,Summer,Rio de Janeiro,Archery,Archery Men's Team,Silver
76575,38864,Zach Garrett,M,21.00,178.00,64.00,United States,USA,2016 Summer,2016,Summer,Rio de Janeiro,Archery,Archery Men's Team,Silver
263042,131620,Taylor Worth,M,25.00,174.00,60.00,Australia,AUS,2016 Summer,2016,Summer,Rio de Janeiro,Archery,Archery Men's Team,Bronze
63167,32384,Brady Lee Ellison,M,27.00,181.00,86.00,United States,USA,2016 Summer,2016,Summer,Rio de Janeiro,Archery,Archery Men's Team,Silver
127977,64487,Ku Bon-Chan,M,23.00,181.00,84.00,South Korea,KOR,2016 Summer,2016,Summer,Rio de Janeiro,Archery,Archery Men's Team,Gold
192202,96508,Alec Potts,M,20.00,185.00,77.00,Australia,AUS,2016 Summer,2016,Summer,Rio de Janeiro,Archery,Archery Men's Team,Bronze


In [75]:
# Dropping duplicate medals for team events

final_result = no_null_olympics_2016.drop_duplicates(subset=['Event', 'NOC', 'Medal'])
final_result.shape

(968, 15)

In [79]:
medal_count_by_NOC_2016 = final_result.NOC.value_counts()
medal_count_by_NOC_2016

USA    121
CHN     70
GBR     66
RUS     56
GER     42
      ... 
NIG      1
POR      1
EST      1
PUR      1
FIN      1
Name: NOC, Length: 86, dtype: int64